# Random Forest

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

data = pd.read_csv("C:\\Users\\racha\\OneDrive\\Desktop\\ddos_data.csv")

data['Time'] = data['Time'].apply(lambda x: x.split(':'))
data['hour'] = data['Time'].apply(lambda x: int(x[0]))
data['minute_int'] = data['Time'].apply(lambda x: int(x[1].split('.')[0]))
data['minute_dec'] = data['Time'].apply(lambda x: int(x[1].split('.')[1]))

data['second'] = data['Time'].apply(lambda x: 
                                     float(x[2]) if len(x) >= 3 else 0.0)

data.drop('Time', axis=1, inplace=True)

X = data[['hour', 'minute_int', 'minute_dec', 'second', 'Source', 'Destination', 'Protocol', 'Length', 'Src port', 'Dest port']]
y = data['Target']

X_encoded = pd.get_dummies(X, columns=['Source', 'Destination','Protocol'])

X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train_encoded, y_train)

y_pred = model.predict(X_test_encoded)
print(classification_report(y_test, y_pred))


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

joblib.dump(model, 'ddos_model_random.pkl')


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4167
           1       1.00      1.00      1.00      8749

    accuracy                           1.00     12916
   macro avg       1.00      1.00      1.00     12916
weighted avg       1.00      1.00      1.00     12916

Accuracy: 0.9999225766491173


['ddos_model_random.pkl']

# Logistic Regression

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

data = pd.read_csv("C:\\Users\\racha\\OneDrive\\Desktop\\ddos_data.csv")

X = data.drop(['Target', 'Time', 'Source', 'Destination', 'Protocol'], axis=1)
X = X.select_dtypes(include=['float64', 'int64'])
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           0       1.00      0.78      0.88      4167
           1       0.91      1.00      0.95      8749

    accuracy                           0.93     12916
   macro avg       0.95      0.89      0.92     12916
weighted avg       0.94      0.93      0.93     12916

Accuracy: 0.9306286776091669


# Capturing Live traffic